# Chapter 5 - The Simple Recurrent Neural Network

_pg. 67-82_

## Prepare the Sample Data

In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing

loc = "/home/dpindk/irlab/big-data-summer-school-2017/COE.csv"
temp = pd.read_csv(loc)
data = temp.drop(temp.columns [[0,1]], axis=1)

y = data["COE$"]

x = data.drop(data.columns[[0,4]], axis=1)
x = x.apply(np.log)
x = pd.concat([x, data["Open?"]], axis=1)

scaler_x = preprocessing.MinMaxScaler(feature_range=(0,1))
x = np.array(x).reshape((len(x), 4))
x = scaler_x.fit_transform(x)

scaler_y = preprocessing.MinMaxScaler(feature_range=(0,1))
y = np.array(y).reshape((len(y), 1))
y = np.log(y)
y = scaler_y.fit_transform(y)

## Follow this Approach to Create Your Train and Test Sets

In [2]:
end = len(x) - 1
learn_end = int(end * 0.954)
x_train = x[0:learn_end - 1, ]
x_test = x[learn_end:end - 1, ]
y_train = y[1:learn_end]
y_test = y[learn_end + 1: end]

x_train = x_train.reshape(x_train.shape + (1,))
x_test = x_test.reshape(x_test.shape + (1,))

In [3]:
print "Shape of x_train is ", x_train.shape

Shape of x_train is  (250, 4, 1)


In [4]:
print "Shape of x_test is ", x_test.shape

Shape of x_test is  (12, 4, 1)


## Import Keras

In [5]:
from keras.models import Sequential
from keras.optimizers import SGD
from keras.layers.core import Dense, Activation
from keras.layers.recurrent import SimpleRNN

Using Theano backend.


## Determine Model Structure

In [6]:
seed = 2016
np.random.seed(seed)
fit1 = Sequential()
# UPDATED
# fit1.add(SimpleRNN(output_dim=8, activation="tanh", input_shape=(4,1)))
fit1.add(SimpleRNN(units=8, activation="tanh", input_shape=(4,1)))
# UPDATED
# fit1.add(Dense(output_dim=1, activation='linear'))
fit1.add(Dense(units=1, activation='linear'))

## Choosing the Moment Value

In [7]:
# uses accelerated gradient descent
sgd = SGD(lr=0.0001, momentum=0.95, nesterov=True)
fit1.compile(loss='mean_squared_error', optimizer=sgd)

## Fitting the Model
The model, with a batch size of 10, is fit over 700 epochs as follows:

In [8]:
%%capture
fit1.fit(x_train, y_train, batch_size=10, epochs=700)

In [9]:
score_train = fit1.evaluate(x_train, y_train, batch_size=10)
score_test = fit1.evaluate(x_test, y_test, batch_size=10)
print "in train MSE = ", round(score_train, 6)
print "in test MSE = ", round(score_test, 6)

10/12 [========================>.....] - ETA: 0sin train MSE =  0.003043
in test MSE =  0.000757


In [10]:
pred1 = fit1.predict(x_test)
print pred1

[[ 0.55883271]
 [ 0.53600138]
 [ 0.53680229]
 [ 0.54093546]
 [ 0.57687551]
 [ 0.59036571]
 [ 0.6088571 ]
 [ 0.58301008]
 [ 0.56355613]
 [ 0.55584341]
 [ 0.58015609]
 [ 0.58528125]]


In [11]:
# apply the inverse transformation to predictions
pred1 = scaler_y.inverse_transform(np.array(pred1).reshape((len(pred1), 1)))
print pred1

[[ 9.50765705]
 [ 9.40135479]
 [ 9.40508461]
 [ 9.42432785]
 [ 9.59166431]
 [ 9.65447426]
 [ 9.74056816]
 [ 9.62022591]
 [ 9.52964878]
 [ 9.49373913]
 [ 9.60693836]
 [ 9.63080025]]


In [12]:
# undo the initial log transformation of the data
pred1 = np.exp(pred1)
print np.rint(pred1)

[[ 13462.]
 [ 12105.]
 [ 12150.]
 [ 12386.]
 [ 14642.]
 [ 15591.]
 [ 16993.]
 [ 15066.]
 [ 13762.]
 [ 13276.]
 [ 14868.]
 [ 15227.]]
